# Function 5 - Chemical Yield

### First Inspection

We are told, according to the function description provided, that the black-box function $f(x_1, x_2, x_3, x_4)$ represents the yield of a chemical process in a factory. Very importantly, we also told that this distribution in typically unimodal with a single peak where yield is maximised. Let us PCA for visualisation,

In [12]:
# Depedencies,
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# SKOPT imports,
from skopt import gp_minimize
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from skopt import Optimizer
from joblib import dump, load

# Loading known evaluations,
X, y = np.load("initial_inputs.npy"), np.load("initial_outputs.npy")

# Inspecting evalutation points,
X

array([[0.19144708, 0.03819337, 0.60741781, 0.41458414],
       [0.75865295, 0.53651774, 0.65600038, 0.36034155],
       [0.43834987, 0.8043397 , 0.21024527, 0.15129482],
       [0.70605083, 0.53419196, 0.26424335, 0.48208755],
       [0.83647799, 0.19360965, 0.6638927 , 0.78564888],
       [0.68343225, 0.11866264, 0.82904591, 0.56757661],
       [0.55362148, 0.66734998, 0.32380582, 0.81486975],
       [0.35235627, 0.32224153, 0.11697937, 0.47311252],
       [0.15378571, 0.72938169, 0.42259844, 0.44307417],
       [0.46344227, 0.63002451, 0.10790646, 0.9576439 ],
       [0.67749115, 0.35850951, 0.47959222, 0.07288048],
       [0.58397341, 0.14724265, 0.34809746, 0.42861465],
       [0.30688872, 0.31687813, 0.62263448, 0.09539906],
       [0.51114177, 0.817957  , 0.72871042, 0.11235362],
       [0.43893338, 0.77409176, 0.37816709, 0.93369621],
       [0.22418902, 0.84648049, 0.87948418, 0.87851568],
       [0.72526172, 0.47987049, 0.08894684, 0.75976022],
       [0.35548161, 0.63961937,

Despite the sparsity of data points, the domain of the black box function seems to be $[0, 1]^4$.

In [21]:
# Inspecting black-box outputs,
y

array([6.44434399e+01, 1.83013796e+01, 1.12939795e-01, 4.21089813e+00,
       2.58370525e+02, 7.84343889e+01, 5.75715369e+01, 1.09571876e+02,
       8.84799176e+00, 2.33223610e+02, 2.44230883e+01, 6.44201468e+01,
       6.34767158e+01, 7.97291299e+01, 3.55806818e+02, 1.08885962e+03,
       2.88667516e+01, 4.51815703e+01, 4.31612757e+02, 9.97233189e+00])

The chemical yield is positive and can vary by numerous magnitudes between each point.

### Optimiser Configuration

From the discription of the black-box function, we know,

- $f(\mathbf{x})$ is likely to be unimodal and have a single global maximum. 
- The black-box function is not noisy (problem discription has not mentioned noise).

However, we do NOT know,

- If black-box function $f(\mathbf{x})$ is very smooth, generally smooth or rough.

As a result, our go-to Matérn 5/2 kernel (with ARD). Assuming that $f(\mathbf{x})$ is unimodal, we should consider exploitation instead of exploration. Therefore, we select PI as our acquistion function. 

[Equation]

Our strategy is to slowly _"climb"_ the peak of the distribution by exploiting large known values of $f(\mathbf{x})$. To be pendantic, a given element of a chosen kernel is given by,

$$
K(\mathbf{x}_i, \mathbf{x}_j)
=
\sigma^2
\frac{2^{1-\nu}}{\Gamma(\nu)}
\left(
\sqrt{2\nu}\, r_{ij}
\right)^{\nu}
K_{\nu}
\left(
\sqrt{2\nu}\, r_{ij}
\right),
$$

$$
r_{ij}
=
\sqrt{
\frac{(x_{i1} - x_{j1})^2}{\ell_1^2}
+
\frac{(x_{i2} - x_{j2})^2}{\ell_2^2}
+
\frac{(x_{i3} - x_{j3})^2}{\ell_3^2}
+
\frac{(x_{i4} - x_{j4})^2}{\ell_4^2}
}.
$$

where we have set the lengthscales as $\ell_1 = \ell_2 = \ell_3 = \ell_4 = 0.1$ and used $\nu = 5/2$ as well as $\sigma = 1$.

### Optimiser Initialisation

In [14]:
"""INITALISING THE OPTIMISATION MODEL."""

# Inputting the given evaluations provided by the problem,
X_supplied = X.tolist()
y_supplied = y.tolist()

"""OPTIMISER SETTINGS."""

# We define the domain of the black-box function (or the range of the parameter values we want to consider),
space = [Real(0, 1, name="x1"),
         Real(0, 1, name="x2"),
         Real(0, 1, name="x3"),
         Real(0, 1, name="x4")
         ]

# Creating the kernel for the GPR,
kernel = ConstantKernel(1.0) * Matern(
    length_scale=(0.1, 0.1, 0.1, 0.1),
    length_scale_bounds=(1e-2, 1.0),
    nu = 5/2)

# GPR settings,
gpr = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=10
)

# Creating optimisier,
opt = Optimizer(
    dimensions=space,
    base_estimator=gpr,
    acq_func="EI",
    acq_func_kwargs={"xi": 0.01},
    random_state=0
)

"""CREATING INTIAL OPTIMISER STATE."""

# Supplying given points to optimiser,
opt.tell(X_supplied, (-np.array(y_supplied)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state (zero-th iteration),
dump(opt, "bayes_opt_state_iter0.joblib")

# Printing point query,
print(f"Point Query: {point_query}")

Point Query: [0.2570963855872633, 0.9158785285501715, 1.0, 1.0]


### Next Query 

In [ ]:
current_query = 1

# Input the new evaluation,
X_new = [[]]
y_new = []

# Loading the previous state of the optimiser,
opt = load(f"bayes_opt_state_iter{current_query - 1}.joblib")

# Supplying the new query to the optimiser,
opt.tell(X_new, (-np.array(y_new)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state,
dump(opt, f"bayes_opt_state_iter{current_query}.joblib")

# Printing point query,
print(f"Point Query {current_query + 1}: {point_query}")

### Update

Week 1: The optimiser sampled a point in close proximity to known point with the highest evaluation value. This is not unexpected as PI is considered to be the most greedy and exploitive of the acquisition functions available in `skopt`. However, it does appear to be too greedy in that we are not "climbing" the peak of the unimodal distribution quickly enough. For this reason, we replace PI with EI and restart the optimiser. We hope that EI remains primariliy exploitative, but is not as greedy such that we can "climb" towards global maximum at a faster pace.